In [10]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,  # 데이터의 제한을 측정하기 어려운 사람, ai간의 대화를 저장
)

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history",  # default=history, history 변수명을 써도 상관없음
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def load_memory(input):  # input 대신 '_' 써도 상관없음
    return memory.load_memory_variables({})["chat_history"]

# RunnablePassthrough: 프롬프트가 format되기 전에 우리가 함수를 실행시키는 걸 허락해줌
# load_memory의 결과값을 chat_history input으로 넣어주라고 알려줄 수 있음
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm 


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


invoke_chain(question="My name is Jun")

content='Nice to meet you, Jun! How can I assist you today?'


In [11]:
invoke_chain(question="What is my name?")

content='Your name is Jun.'
